In [12]:
# DATA READ - REYNOLDS AND VELOCITY VALUES
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#import data

ANNDF = pd.read_excel('fxReynoldsV3.xlsx')

X = ANNDF.iloc[:,[1,3]].values
Y = ANNDF.iloc[:, 2].values.reshape(-1,1)

xtrain, xtest, ytrain,  ytest = train_test_split(X, Y, train_size = 0.75)

scalerx = StandardScaler()
scalery = StandardScaler()

Xtrain = scalerx.fit_transform(xtrain)
Ytrain = scalery.fit_transform(ytrain)



mlp = MLPRegressor(max_iter=1000)



In [15]:

# Define the KFold cross-validation
kf = KFold(n_splits=10, shuffle=True)

# Define the parameter grid
parameters = {
    'hidden_layer_sizes': [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (8, 8)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam']
}

# Define GridSearchCV
gsearchcv = GridSearchCV(estimator=mlp, param_grid=parameters, cv=kf)

# Fit the model
gsearchcv.fit(Xtrain, Ytrain)

# Get best score and parameters
bestac = gsearchcv.best_score_ * 100  # Convert to percentage
bestparam = gsearchcv.best_params_

print('Best Accuracy: {:.2f} %'.format(bestac))
print('Best Parameters:', bestparam)


c:\Users\T2F\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\T2F\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\T2F\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\T2F\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was pass

Best Accuracy: 99.01 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (8, 8), 'solver': 'adam'}


c:\Users\T2F\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
z = (scalerx.transform(xtest))
print(scalerx.inverse_transform(z))

[[2.47398332e+03 4.16200000e-01]
 [4.08938217e+03 3.32600000e-01]
 [2.86326693e+03 3.74500000e-01]
 ...
 [2.71201612e+03 3.99600000e-01]
 [1.75962181e+03 3.32600000e-01]
 [2.51949204e+03 3.74500000e-01]]


In [46]:
input_data = np.array([[1843, 0.4331]])

# Transformar os dados de entrada usando o scaler treinado
input_data_scaled = scalerx.transform(input_data)

# Fazer a previsão com o modelo treinado
predicted_scaled = gsearchcv.predict(input_data_scaled)

# Inverter a transformação do output (scaling)
predicted = scalery.inverse_transform(predicted_scaled.reshape(-1, 1))

Xtest_scaled = scalerx.transform(xtest)

ypred = gsearchcv.predict(Xtest_scaled).reshape(-1,1)

ypred_or = scalery.inverse_transform(ypred)

# Mostrar o resultado
print('Predicted value for [2000, 0.24]:', predicted[0][0])


Predicted value for [2000, 0.24]: 0.4770637145781287


In [47]:
from pysr import PySRRegressor

# Exemplo de ajuste simbólico para uma superfície
model = PySRRegressor(
    niterations=1000,
    binary_operators=["+", "*", "-", "/"],
    unary_operators=["square", "exp", "log"],
)

model.fit(X, ypred_or)
print("Equações encontradas para a superfície:", model.equations_)

[juliapkg] Found dependencies: c:\users\t2f\anaconda3\lib\site-packages\juliapkg\juliapkg.json
[juliapkg] Found dependencies: c:\users\t2f\anaconda3\lib\site-packages\pysr\juliapkg.json
[juliapkg] Found dependencies: c:\users\t2f\anaconda3\lib\site-packages\juliacall\juliapkg.json
[juliapkg] Locating Julia ~1.6.7, ~1.7, ~1.8, ~1.9, =1.10.0, ~1.10.3
[juliapkg] Querying Julia versions from https://julialang-s3.julialang.org/bin/versions.json
[juliapkg] WARNING: About to install Julia 1.10.4 to C:\Users\T2F\anaconda3\julia_env\pyjuliapkg\install.
[juliapkg]   If you use juliapkg in more than one environment, you are likely to
[juliapkg]   have Julia installed in multiple locations. It is recommended to
[juliapkg]   install JuliaUp (https://github.com/JuliaLang/juliaup) or Julia
[juliapkg]   (https://julialang.org/downloads) yourself.
[juliapkg] Downloading Julia from https://julialang-s3.julialang.org/bin/winnt/x64/1.10/julia-1.10.4-win64.zip
             download complete
[juliapkg] Veri

: 